In [ ]:
from matplotlib import pyplot as plt
from collections import Counter, defaultdict
from IPython.display import display, HTML
import seaborn as sns
import pandas as pd
import psycopg2
import requests
import pickle
import random
import os
import re

import warnings

warnings.filterwarnings("ignore")

%load_ext nb_black

In [ ]:
def connect():
    conn = psycopg2.connect(
        user="postgres",
        password=os.environ.get("POSTGRES_PASS", ""),
        host="localhost",
        port=5432,
        database="venmo",
    )
    return conn

In [ ]:
if os.path.exists("trans-clf-dataset.pkl"):
    print('loaded labels.')
    with open("trans-clf-dataset.pkl", "rb") as fp:
        labels = pickle.load(fp)
else:
    labels = {}

conn = connect()
loop = True
while loop:
    with conn.cursor(name="exploration") as cursor:
        cursor.itersize = 2000
        cursor.execute("SELECT * FROM transactions ORDER BY RANDOM() LIMIT 50")
        for i, row in enumerate(cursor):
            raw_msg = row[1]
            if raw_msg in labels:
                continue
            resp = input(raw_msg + " ")
            if "e" in resp:
                loop = False
                break
            elif "y" in resp:
                labels[raw_msg] = True
            else:
                labels[raw_msg] = False
        print("saving...")
        with open("trans-clf-dataset.pkl", "wb") as fp:
            pickle.dump(labels, fp)

conn.close()

In [ ]:
labels